In [1]:
# https://github.com/geopandas/geopandas/issues/830
import os
os.environ["PROJ_LIB"] = "C:\\Users\\flavi\\Anaconda3\\Library\\share" #windows

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from lxml import objectify
import csv

In [2]:
# data_frame = pd.DataFrame.from_csv('yelp_places.csv')

# data_frame.head()

yelp_places_geo = gpd.read_file('Dataset/Processed/yelp_places_arts.shp', encoding='utf-8')

In [3]:
mainXml = objectify.parse('Dataset/Original/London_division_2500-150.xml')
lxmlRoot = mainXml.getroot()
allPlaces = []
for location in lxmlRoot.iterchildren():
    place = {}
    place['id'] = int(location.id)
    place['name'] = str(location.name)
    
    shape_points = []
    for point in location.shape.iterchildren():
        shape_points.append((point.xCoord, point.yCoord))
    polygon_geom = Polygon(shape_points)
    place['polygon'] = polygon_geom
    
    allPlaces.append(place)

division_df = pd.DataFrame.from_records(allPlaces)
division_df.head()

,id,name,polygon
0,1,Trafalgar Square,"POLYGON ((-0.1304619 51.50952, -0.13048266 51...."
1,2,Big Ben,"POLYGON ((-0.12498988 51.503284, -0.12284501 5..."
2,3,Parliament Square,"POLYGON ((-0.12562872 51.502274, -0.12844071 5..."
3,4,London Eye,"POLYGON ((-0.12235283 51.503895, -0.12092333 5..."
4,5,Tate Modern,"POLYGON ((-0.0996684 51.51117, -0.101172544 51..."


In [4]:
division_geo = gpd.GeoDataFrame(division_df, geometry='polygon')
division_geo.crs = {'init' :'epsg:4326'} # lat long projection
division_geo['area'] = division_geo['polygon'].to_crs({'init': 'epsg:3395'}).map(lambda p: p.area / 10**6) # Km 2
# division_geo.to_file('Dataset/Processed/London_division.shp', driver="ESRI Shapefile")
division_geo.head()

,id,name,polygon,area
0,1,Trafalgar Square,"POLYGON ((-0.1304619 51.50952, -0.13048266 51....",0.277425
1,2,Big Ben,"POLYGON ((-0.12498988 51.503284, -0.12284501 5...",0.399994
2,3,Parliament Square,"POLYGON ((-0.12562872 51.502274, -0.12844071 5...",0.236767
3,4,London Eye,"POLYGON ((-0.12235283 51.503895, -0.12092333 5...",0.425408
4,5,Tate Modern,"POLYGON ((-0.0996684 51.51117, -0.101172544 51...",0.686323


In [34]:
visitors = pd.read_csv('Dataset/Processed/Visitors-Visitors_per_week_hour.csv')
joined_visitors = pd.sjoin(division_geo, yelp_places_geo, how="inner", op='intersects')
# visitors['visitors_per_km_2'] = 
visitors.head()
# visitors['N visitors total']

,id,Name,N_visits,week hour=1;00: N visitors by week hours,week hour=1;01: N visitors by week hours,week hour=1;02: N visitors by week hours,week hour=1;03: N visitors by week hours,week hour=1;04: N visitors by week hours,week hour=1;05: N visitors by week hours,week hour=1;06: N visitors by week hours,...,week hour=7;15: N visitors by week hours,week hour=7;16: N visitors by week hours,week hour=7;17: N visitors by week hours,week hour=7;18: N visitors by week hours,week hour=7;19: N visitors by week hours,week hour=7;20: N visitors by week hours,week hour=7;21: N visitors by week hours,week hour=7;22: N visitors by week hours,week hour=7;23: N visitors by week hours,N visitors total
0,1,1,16615,106,97,78,52,51,40,37,...,283,259,258,255,212,185,155,146,127,15201
1,2,2,11042,99,77,33,18,14,18,12,...,106,99,111,106,108,111,109,112,95,10186
2,3,3,4583,26,15,17,11,9,9,8,...,54,51,58,47,44,39,32,36,18,4323
3,4,4,12556,111,77,48,35,26,31,25,...,205,168,153,137,141,172,162,123,129,11558
4,5,5,10098,52,40,29,29,23,25,22,...,207,206,189,162,115,94,85,83,66,9433


In [5]:
intersection = gpd.sjoin(division_geo, yelp_places_geo, how="inner", op='intersects')
intersection.head()

,id_left,name_left,polygon,area,index_right,category,id_right,lat,lng,name_right,price_leve,rating,review_cou
0,1,Trafalgar Square,"POLYGON ((-0.1304619 51.50952, -0.13048266 51....",0.277425,5,galleries,uNBw-V4ZYn-cXt4fc9Jw9Q,51.509540,-0.127580,National Portrait Gallery,1,4.5,189
3,4,London Eye,"POLYGON ((-0.12235283 51.503895, -0.12092333 5...",0.425408,3,arcades,IAIG4QvARG-qRDUcOjOmow,51.501313,-0.119495,Namco Funscape,3,3.0,23
3,4,London Eye,"POLYGON ((-0.12235283 51.503895, -0.12092333 5...",0.425408,412,musicvenues,VBO0oSm1CKrIxhv1drg_PA,51.502998,-0.115822,South Bank Centre,2,4.0,5
4,5,Tate Modern,"POLYGON ((-0.0996684 51.51117, -0.101172544 51...",0.686323,121,galleries,UsjDCA7glkknvPQccywfTg,51.508064,-0.101736,Bankside Gallery,3,4.0,5
4,5,Tate Modern,"POLYGON ((-0.0996684 51.51117, -0.101172544 51...",0.686323,300,musicvenues,wD0BMblDiufinDO24o8x8Q,51.508113,-0.097109,Shakespeare's Globe Theatre,2,4.5,238


In [6]:
business_count = intersection[['id_left', 'index_right']].groupby(['id_left']).agg(['count'])
business_count['area'] = division_geo['area']
business_count.to_csv('yelp_business_count_arts.csv')
business_count.head()

,index_right,area
,count,
id_left,,
1,1,0.399994
4,2,0.686323
5,3,0.326591
6,13,0.493871
7,2,1.157639


In [14]:
plt = intersection.plot(column='category')
plt.show()

In [7]:
def count_per_area(area_list):
    return len(area_list)/list(area_list)[0]

In [8]:
proportional_by_id = intersection.pivot_table(index='id_left', columns='category', values='area', fill_value=0, aggfunc=count_per_area)
proportional_by_id.to_csv('yelp_categories_per_division_arts.csv')
proportional_by_id.head()

category,arcades,cabaret,casinos,culturalcenter,festivals,galleries,jazzandblues,movietheaters,museums,musicvenues,opera,psychic_astrology,social_clubs,theater,wineries
id_left,,,,,,,,,,,,,,,
1,0.000000,0.000000,0.0,0.0,0.000000,3.604582,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0
4,2.350682,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,2.350682,0.0,0.0,0.0,0.000000,0.0
5,0.000000,0.000000,0.0,0.0,0.000000,1.457039,0.000000,0.0,0.0,1.457039,0.0,0.0,0.0,1.457039,0.0
6,0.000000,3.061933,0.0,0.0,0.000000,0.000000,6.123866,0.0,0.0,18.371597,0.0,0.0,0.0,12.247731,0.0
7,0.000000,0.000000,0.0,0.0,2.024821,0.000000,0.000000,0.0,0.0,2.024821,0.0,0.0,0.0,0.000000,0.0


In [9]:
category_count = intersection.pivot_table(index='id_left', columns='category', values='area', fill_value=0, aggfunc=lambda x: len(x))
category_count.to_csv('yelp_categories_count_arts.csv')
category_count.head()

category,arcades,cabaret,casinos,culturalcenter,festivals,galleries,jazzandblues,movietheaters,museums,musicvenues,opera,psychic_astrology,social_clubs,theater,wineries
id_left,,,,,,,,,,,,,,,
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0
6,0,1,0,0,0,0,2,0,0,6,0,0,0,4,0
7,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0


In [29]:
price_level_by_id = intersection.pivot_table(index='id_left', columns='price_leve', values='area', fill_value=0, aggfunc=count_per_area)
price_level_by_id.to_csv('yelp_prices_per_division.csv')
price_level_by_id.head()

price_leve,1,2,3,4
id_left,,,,
1,36.045821,43.254985,46.859567,3.604582
2,5.000072,0.000000,2.500036,0.000000
3,12.670659,25.341318,29.564871,21.117765
4,9.402729,30.558869,18.805458,0.000000
5,14.570395,32.054869,5.828158,1.457039


In [10]:
price_level_count = intersection.pivot_table(index='id_left', columns='price_leve', values='area', fill_value=0, aggfunc=lambda x: len(x))
price_level_count.to_csv('yelp_price_count_division_arts.csv')
price_level_count.head()

price_leve,1,2,3,4
id_left,,,,
1,1,0,0,0
4,0,1,1,0
5,0,2,1,0
6,2,7,2,2
7,0,2,0,0


In [30]:
rating_by_id = intersection.pivot_table(index='id_left', columns='rating', values='area', fill_value=0, aggfunc=count_per_area)
rating_by_id.to_csv('yelp_rating_per_division.csv')
rating_by_id.head()

rating,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
id_left,,,,,,,,,
1,0.0,0.000000,7.209164,14.418328,18.022910,32.441239,39.650403,7.209164,10.813746
2,0.0,0.000000,0.000000,2.500036,2.500036,0.000000,0.000000,0.000000,2.500036
3,0.0,8.447106,8.447106,4.223553,0.000000,25.341318,12.670659,12.670659,16.894212
4,0.0,0.000000,2.350682,4.701364,16.454776,16.454776,14.104093,4.701364,0.000000
5,0.0,0.000000,1.457039,1.457039,1.457039,14.570395,26.226711,8.742237,0.000000


In [11]:
ratings_count = intersection.pivot_table(index='id_left', columns='rating', values='area', fill_value=0, aggfunc=lambda x: len(x))
ratings_count.to_csv('yelp_rating_count_division_arts.csv')
ratings_count.head()

rating,1.0,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0
id_left,,,,,,,,,
1,0,0,0,0,0,0,0,1,0
4,0,0,0,0,1,0,1,0,0
5,0,0,0,0,0,0,1,2,0
6,0,0,0,1,1,0,8,3,0
7,0,0,0,0,0,0,2,0,0
